In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
.builder \
.appName("App") \
.master("local[*]") \
.enableHiveSupport() \
.getOrCreate()

In [3]:
spark

In [4]:
spark.sql("CREATE DATABASE IF NOT EXISTS ds")

DataFrame[]

In [8]:
spark.sql( """
CREATE TABLE IF NOT EXISTS ds.md_account_d (
    data_actual_date DATE,
    data_actual_end_date DATE,
    account_rk DECIMAL,
    account_number STRING,
    char_type STRING,
    currency_rk DECIMAL,
    currency_code STRING
)
STORED AS PARQUET
"""
)

DataFrame[]

In [26]:
data = spark.read.option("delimiter", ";").csv("/home/jovyan/md_account_d.csv", header=True)

In [27]:
data.show()

+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
|DATA_ACTUAL_DATE|DATA_ACTUAL_END_DATE|ACCOUNT_RK|      ACCOUNT_NUMBER|CHAR_TYPE|CURRENCY_RK|CURRENCY_CODE|
+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
|      2018-01-01|          2018-01-31|  36237725|30425840700000583001|        А|         35|          840|
|      2018-01-01|          2018-01-31|     24656|30114840700000770002|        А|         35|          840|
|      2018-01-01|          2018-01-31|  18849846|30109810500000435003|        П|         34|          643|
|      2018-01-01|          2018-01-31|   1972647|30111810700000908001|        П|         34|          643|
|      2018-01-01|          2018-01-31|  34157174|30424810100000583001|        А|         34|          643|
|      2018-01-01|          2018-01-31|  48829156|30425810700000583002|        А|         34|          643|
|      2018-01-01|          

In [31]:
data.printSchema()

root
 |-- DATA_ACTUAL_DATE: string (nullable = true)
 |-- DATA_ACTUAL_END_DATE: string (nullable = true)
 |-- ACCOUNT_RK: string (nullable = true)
 |-- ACCOUNT_NUMBER: string (nullable = true)
 |-- CHAR_TYPE: string (nullable = true)
 |-- CURRENCY_RK: string (nullable = true)
 |-- CURRENCY_CODE: string (nullable = true)



In [32]:
data.createOrReplaceTempView("data")

In [49]:
data_new = spark.sql(
    """
SELECT to_date(DATA_ACTUAL_DATE) as DATA_ACTUAL_DATE,
    to_date(DATA_ACTUAL_END_DATE) as DATA_ACTUAL_END_DATE,
    CAST(ACCOUNT_RK AS DECIMAL),
    ACCOUNT_NUMBER,
    CHAR_TYPE,
    CAST(CURRENCY_RK AS DECIMAL),
    CURRENCY_CODE
FROM data
"""
)

In [50]:
data_new.show()

+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
|DATA_ACTUAL_DATE|DATA_ACTUAL_END_DATE|ACCOUNT_RK|      ACCOUNT_NUMBER|CHAR_TYPE|CURRENCY_RK|CURRENCY_CODE|
+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
|      2018-01-01|          2018-01-31|  36237725|30425840700000583001|        А|         35|          840|
|      2018-01-01|          2018-01-31|     24656|30114840700000770002|        А|         35|          840|
|      2018-01-01|          2018-01-31|  18849846|30109810500000435003|        П|         34|          643|
|      2018-01-01|          2018-01-31|   1972647|30111810700000908001|        П|         34|          643|
|      2018-01-01|          2018-01-31|  34157174|30424810100000583001|        А|         34|          643|
|      2018-01-01|          2018-01-31|  48829156|30425810700000583002|        А|         34|          643|
|      2018-01-01|          

In [51]:
data_new.write.insertInto("ds.md_account_d")